In [ ]:
import pyspark

Like before, the first step is to initialize a spark context:

In [ ]:
sc = pyspark.SparkContext()

Now we can initialize a SQL Context passing the <code>sc</code> object to the <code>SQLContext</code> function. The convention when initializing a <code>SQLContext</code> is to pass it to a variable called <code>spark</code>, though you will also see many examples where this variable is called <code>sql</code> or <code>SQLContext</code>.

In [ ]:
from pyspark.sql import SQLContext

spark = SQLContext(sc)

The Spark SQL API, unlike the RDD API, has a method to read CSV files directly. Let's use it to load our museum data from the RDD API notebook. An alterntive syntax for the command below is <code>spark.read.format("csv").load('MetObjects.csv')</code>. Reading this you might be asking yourself if the SparkSQL API has methods to read other formats directly... and the answer is yes, it does. We will mostly focus on CSV today, but you can find a complete list of supported data sources here: https://spark.apache.org/docs/latest/sql-data-sources.html

In [ ]:
museum_data = spark.

The command above loaded our CSV into a data structure called a DataFrame. If you are familiar with R or Pandas, you will be right at home with the SparkSQL API! 

The <code>head</code> method shows you the first row of a DataFrame. Notice the word "Row" on the output: a DataFrame is an RDD where the elements are objects of the <code>Row</code> class!

In [ ]:
museum_data.

Another way of peeking into a DataFrame is the <code>show</code> method. This prints your DataFrame in a way analogous to how you'd get an output from a Relational Database Management Software (RDBMS) on the command line.

In [ ]:
museum_data.

A third option is using the method <code>toPandas</code>. As the name says, this will bring your DataFrame back to the Driver and convert it into a Pandas DataFrame. Notice the <code>limit</code> method called just before <code>toPandas</code>: since this is bringing data back from the Cluster to the Driver, you need to make sure you are not bringing too much back!

In [ ]:
museum_data.

Now let's look into how to maipulate data with the SparkSQL API. We'll start with the DataFrames API component. It looks just like SQL, but instead of writing statements, you will call methods just like we did with the RDD API. For example, you can select a single column out of a DataFrame where a condition based on another column is satisfied, then count the number of rows returned:

In [ ]:
museum_data.

Also like the RDD API, you can use the <code>take</code> method to bring a few rows back to the Driver and print the results:

In [ ]:
museum_data.

Similarly to how you'd do it using SQL, you can use the <code>distinct</code> method to return unique instances out of one or more columns:

In [ ]:
museum_data.

In [ ]:
museum_data.

The <code>groupBy</code> method works like the "GROUP BY" SQL clause: use it to apply aggregations to your data based on one or more columns:

In [ ]:
museum_data.

In [ ]:
museum_data.

The <code>OrderBy</code> method works like the "ORDER BY" SQL statement: notice that its position in the chain of methods we call is analogous to where you'd place an ORDER BY statement in a SQL query!

In [ ]:
museum_data.

In [ ]:
museum_data.

Now, you must be wondering why we chose this Museum dataset for the workshop. The line above is why: this dataset is incredibly broken! Notice on the block above how none of the outputs look like an actual Artist Display Name. The next line gives us an idea of what is going on here - there are too many quotes in some of the coulmns! Look at the "Title" column to see what we mean.

In [ ]:
museum_data.

We will try to use Spark to fix this problem later. For now, let's demonstrate a couple more things you can do with the DataFrames API while also demonstrating other ways in which this dataset is broken... here we use the <code>like</code> method to retrieve instances where the Artist's Nationality is an URL. Notice the SQL syntax in the argument of <code>like</code>: the wildcard character to tell Spark we are looking for the pattern "http followed by anything" is "%".

In [ ]:
museum_data.

The <code>where</code> method returns rows where whatever is passed into it as an argument is true. To get an equivalent to SQL's "NOT LIKE" we negate the output of <code>like</code> with a "~" (tilda). Notice how there are outputs that don't make sense here too!

In [ ]:
museum_data.

The next line will give us an idea of what is going on here: look at the very first column, "Object Number". Its value is 'white cedar"'. That lone quote at the end tells us this is probably the end of a larger quoted line and it ended up here because the line contains a new-line character (\n)! 

In [ ]:
museum_data.

We will look into how to fix those problems shortly. Before we do that though, let's look at the reason why this API is called SparkSQL in the first place. Using the <code>registerTempTable</code> method you can turn your DataFrame into a full-fledged table and use actual SQL to query it!

In [ ]:
museum_data.

In [ ]:
spark.

In [ ]:
spark.

In [ ]:
spark.

In [ ]:
spark.

You can add multiple tables to your active Spark SQLContext and work on them as you would on a Relational Database - Join tables, Union tables... To see what tables are current registered in your SQLContext, use the <code>tableNames</code> method:

In [ ]:
spark.

We had seen how to register tables with <code>registerTempTable</code>. To remove tables from the session, use <code>dropTempTable</code>:

In [ ]:
spark.

Whether your DataFrame is registered as a Table in the session, you can see its column names by checking the <code>columns</code> attribute:

In [ ]:
museum_data.

And you can remove columns from a DataFrame using the <code>drop</code> method. Adding columns to spark DataFrames is not as easy and we will see one way to do it later!

In [ ]:
museum_data.

If you are familiar with SQL, you must be feeling pretty at home right now with the SparkSQL API. However, you might be asking yourself... how you handle column types? In other words, how can I work in a Database where the tables have no schema?

It turns out Tables and DataFrames in Spark do have schemas! You can check the <code>schema</code> attriubte:

In [ ]:
museum_data.

In our case, wee did not specify what the schema of the DataFrame should be when we loaded the source CSV file, so Spark went ahead and read everything as Strings.

As it turns out, you can actually impose schemas on Spark DataFrames before you start populating them with data, in a way analogous to how you write a DDL statement to define a Table's schema in SQL.

Below you will see an example of what the schema of our museum data could look like. The following entities are worth of note: <code>StructType</code>, <code>StructField</code> and the different data Types. In SparkSQL, the <code>StructType</code> class defines what a Row will contain, and the <code>StructField</code> class defines what the columns in that row will look like.

The order of the StructFields in a StructType must match what is in the dataset you will import into your DataFrame.

In [ ]:
from pyspark.sql.types import *

museum_schema = StructType([StructField('Object Number',StringType()), 
                     StructField('Is Highlight',     BooleanType()), 
                     StructField('Is Timeline Work', BooleanType()), 
                     StructField('Is Public Domain', BooleanType()),
                     StructField('Object ID',        IntegerType()),
                     StructField('Gallery Number',   IntegerType()),
                     StructField('Department',       StringType()),
                     StructField('AccessionYear',    IntegerType()),
                     StructField('ObjectName',       StringType()),
                     StructField('Ttile ',           StringType()),
                     StructField('Culture',          StringType()),
                     StructField('Period',           StringType()),
                     StructField('Dynasty',          StringType()),
                     StructField('Reign',            StringType()),
                     StructField('Portfolio',        StringType()),
                     StructField('Constituent ID',   IntegerType()),
                     StructField('Artist Role',      StringType()),
                     StructField('Artist Prefix',    StringType()),
                     StructField('Artist Display Name',StringType()),
                     StructField('Artist Display Bio',StringType()),
                     StructField('Artist Suffix',    StringType()),
                     StructField('Artist Alpha Sort',StringType()),
                     StructField('Artist Nationality',StringType()),
                     StructField('Artist Begin Date',IntegerType()),
                     StructField('Artist End Date',  IntegerType()),
                     StructField('Artist Gender',    StringType()),
                     StructField('Artist ULAN URL',  StringType()),
                     StructField('Artist Wikidata URL',StringType()),
                     StructField('Object Date',      DateType()),
                     StructField('Object Begin Date',IntegerType()),
                     StructField('Object End Date',  IntegerType()),
                     StructField('Object Display Name',StringType()),
                     StructField('Medium',           StringType()),
                     StructField('Dimensions',       StringType()),
                     StructField('Credit Line',      StringType()),
                     StructField('Geography Type',   StringType()),
                     StructField('City',             StringType()),
                     StructField('State',            StringType()),
                     StructField('County',           StringType()),
                     StructField('Country',          StringType()),
                     StructField('Region',           StringType()),
                     StructField('Subregion',        StringType()),
                     StructField('Locale',           StringType()),
                     StructField('Locus',            StringType()),
                     StructField('Excavation',       StringType()),
                     StructField('River',            StringType()),
                     StructField('Classification',   StringType()),
                     StructField('Rights and Reproduction',StringType()),
                     StructField('Link Resource',    StringType()),
                     StructField('Object Wikidata URL',StringType()),
                     StructField('Metadata Date',    DateType()),
                     StructField('Repository',       StringType()),
                     StructField('Tags',             StringType()),
                     StructField('Tags AAT URL',     StringType()),
                     StructField('Tags Wikidata URL',StringType())
                    ])

Now let's try importing our CSV file and impose the schema above schema on it. To do so, we'll add the "schema" parameter to our call to the <code>csv</code> method we had used before:

In [ ]:
museum_data = 

museum_data.head()

All of our data got loaded as <code>None</code>, which you can think of as the Python equivalent of a SQL <code>NULL</code>. We can confirm that it was indeed **all** of our data that gor loaded as NULLs running the line below. The <code>dropna</code> method, used without any arguments like we do below, filters out all rows containing NULLs from our DataFrame: 

In [ ]:
museum_data.

So what happened here? Well, we know the dataset is broken and that there are likely lots of rows that won't possibly fit into the schema we specified. As it turns out, SparkSQL's default behaviour when reading data, is to fail when it comes across malformed inputs!

We can change that default behaviour by adding an option to drop malformed inputs and keep the good ones to our CSV reader:

In [ ]:
museum_data = 

museum_data.count()

Looks like there is a whole lot of inputs that won't fit in our schema...

In general, enforcing schemas in SparkSQL is a good idea when you know you can trust the source to be **mostly** clean, i.e., that the number of columns and their types will mostly match the schema you intend to enforce. Having a well-defined schema in SparkSQL will enable you to use a number of typed built-in functions directly, without having to convert between types or wrangling data just so you can use a certain function. A notable example of when this comes in handy is whenever you are dealing with Dates or Timestamps.

Since we know our dataset is far from clean, we are better off not enforcing a schema and letting Spark decide instead. We add the option "inferschema" to tell Spark we'd like it to try to figure out from the data what the schema is. **Spoiler alert:** Spark does not infer schemas from messy data very reliably.

In [ ]:
museum_data = 
museum_data.count()

Better, but still pretty far from the over 500k lines our CSV file actually has.

We had identified two problems with our data before:

1. Too many quotes in some lines.
2. Line breaks inside of quoted lines.

Let's try to solve these two problems now, and see if that means we can get more lines. First we will use the "multiline" option to let Spark know that there are some line breaks **in the fields** of our CSV file and they do not mean the next line is a different row of the dataset:

In [ ]:
museum_data = 
museum_data.count()

Getting better! Let's see if this solved our problem with URLs showing up where Artists' nationalities should have been:

In [ ]:
museum_data.

Looks like at least there are no longer URLs in the Artist Nationality field. Let's see if those URLs are in the right field now:

In [ ]:
museum_data.where(museum_data['Tags AAT URL']=='http://vocab.getty.edu/page/aat/300266506|http://vocab.getty.edu/page/aat/300386951|http://vocab.getty.edu/page/aat/300132410').take(1)

Bingo! Now let's see if we can solve the too-many-quotes problem. We do this by adding the "escape" option, indicating that any occurences of the " character inside a field, is to be read as a simple character and not a quote in the CSV-altering sense:

In [ ]:
museum_data = 
museum_data.count()

Got an extra 10k lines, not bad! Let's check what one of our previously broken lines look like now:

In [ ]:
museum_data.

In [ ]:
museum_data.

The key takeaway here is that many data quality problems can be solved in Spark at load time. This is true of most other data formats supported by Spark too! So if you come across problems like the ones we've seen here where records are potentially malformed, a good first step is always to use one of the <code>read</code> methods and their options to solve them. For more on these methods and their options, see here: https://spark.apache.org/docs/latest/sql-data-sources.html

There are many other problems like these left in this dataset and you are welcome to try and find them. They should be possible to fix by adding options to the CSV reader like we did above.

If however you can't solve a problem with one of the <code>read</code> methods, or if you need to transform a field that got loaded successfully (but not correctly) or if you need to add a new column to your DataFrame based on existing columns, a powerful tool at your disposal is... the RDD API!

You can extract an RDD from a DataFrame by reading the <code>rdd</code> attribute like this:

In [ ]:
museum_rdd = museum_data.rdd

A DataFrame, however, is an RDD where the elements are objects of the class Row as we've seen before:

In [ ]:
museum_rdd.take(1)

This isn't very useful outside of the SparkSQL API, so we convert all elements to good-old Python <code>lists</code> next: 

In [ ]:
museum_rdd.

Now, remember from before how we said that adding columns to DataFrames was not as straightforward as it sounds? There are actually many ways to do it, none being particularly easy or obvious to figure out. The one we recommend for the flexibility it gives the programmer is using the RDD API. On the next line, we will append a '1' to each element of our RDD if the Artist's Nationality is "Canadian", a '0' if the artist is not Canadian, and an 'NA' if the artist's nationality is NULL:

In [ ]:
museum_rdd = museum_rdd.

Now let's create a Python list containing the original column names from our DataFrame, plus a new column name: 'Is Canadian'.

In [ ]:
column_names = 

Finally, we convert the RDD back to a DataFrame. Passing our list of column names as an argument tells Spark that this should be the header of the new DataFrame. The second argument tells Spark how much of the RDD it should look at in order to try to figure out the schema of the DataFrame. If your dataset is massive, even 1% (0.01) might be too much and take a long time to run!

In [ ]:
museum_data = 

In [ ]:
museum_data.take(1)

This concludes our introduction to the SparkSQL API. This API is extremely powerful and there is a lot more that you can do with it than we covered 